In [ ]:
#Connect to drive
from google.colab import drive
drive.mount('/content/gdrive')
!ls gdrive/MyDrive/dlvr/

Mounted at /content/gdrive
 AliScrape.py   scrape2.json   scrape5.json   scrape8.json	   scrape.json
 mergeData.py   scrape3.json   scrape6.json  'scrape9 copy.json'   splitToDo.py
 scrape1.json   scrape4.json   scrape7.json   scrape9.json


In [ ]:
!pip install parsel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Scraper

In [ ]:
import requests
import json
from parsel import Selector
from bs4 import BeautifulSoup
from random import randint
from time import sleep
import random

def getCatPage(url, page):
    lis = url.split("/")
    querystring = {"trafficChannel":"main","catName":lis[5].split(".")[0],"CatId":lis[4],"ltype":"wholesale","SortType":"total_tranpro_desc","page":str(page),"groupsort":"1","isrefine":"y"}

    payload = ""
    headers = {
        "cookie": "ali_apache_id=33.3.24.190.164855992375.189351.0; XSRF-TOKEN=cebdb283-e098-462b-b5ca-8e7d56368f8a; intl_locale=en_US; xman_f=T00k0g+45+kVyCBXFo+ZGxD4zF9tzVgSvWgXd6Xs6lO7PToQcZ9cOUzrlujs980cc0aNf8b7EtN8yHSnFx4i+JKTnNxoZtVNEoKtF7UwYYJim3Dknb8ZAw==; aep_usuc_f=site=glo&c_tp=DKK&ups_d=0|0|0|0&ups_u_t=1664731016912&region=DK&b_locale=en_US&ae_u_p_s=1; acs_usuc_t=x_csrf=1b0xyd1bhfel9&acs_rt=c8e9ec21ee7443f5a0b657a018169e3f; xman_t=g/CiU0iYUAq/arD8sEi676y5pnqn5FXpwnODC+lXbwNOKECkJCe1oxojkz6oDABz; ali_apache_track=; ali_apache_tracktmp=; e_id=pt90; _m_h5_tk=cbdd747895668e3e0f3b4ff94aa2acb8_1663589691466; _m_h5_tk_enc=b52d006eda32cbd4eefda76070103880; AKA_A2=A; xman_us_f=x_locale=en_US&x_l=0&x_c_chg=0&x_as_i=%7B%22cookieCacheEffectTime%22%3A1663588459153%2C%22isCookieCache%22%3A%22Y%22%2C%22ms%22%3A%220%22%7D&acs_rt=c8e9ec21ee7443f5a0b657a018169e3f; aep_history=keywords%5E%0Akeywords%09%0A%0Aproduct_selloffer%5E%0Aproduct_selloffer%091005004727409574%091005003490461568%094000094534783%094001082200193%0932946165416%091005004689057643%091005004023414962%091005002227573759; JSESSIONID=6881F5C023F49E99EEDEFF91E90204AF; x5secdata=xbc05a7d12dcc09d7ef5da8a91f12b32d41663588244a976937840a-187012451abazd2daa__bx__www.aliexpress.com%3A443%2Fglosearch%2Fapi%2Fproduct; intl_common_forever=BuKhQe7uvgN43rzuNFjDpz6xTVZs880vAT4+v72L0mKtZqg1KIvGbg==; tfstk=cOk1BpVMiu4FlrdU6Ste7lfV_oyRZ9LQ0OZSfeR7nf2OUon1ioXzPcS8oZyyTk1..; l=eBxlqkMILY9Nz14bBOfZnurza77TKIRfguPzaNbMiOCP9L565UrAW6oT-ALBCnMNnstHR3PgPMOyB5YU0y4oh1ZN5AVBDpB3wdLh.; isg=BK-vdts1EA7aTRVZtlo8_slbPsq5VAN2q5aN38E8BZ4lEM4SySUnx6mCkhguaNvu",
        "authority": "FILL",
        "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8",
        "accept-language": "en-US,en;q=0.5",
        "cache-control": "max-age=0",
        "sec-fetch-dest": "document",
        "sec-fetch-mode": "navigate",
        "sec-fetch-site": "same-origin",
        "sec-fetch-user": "?1",
        "sec-gpc": "1",
        "upgrade-insecure-requests": "1",
        "user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36"
    }

    response = requests.request("GET", url, data=payload, headers=headers, params=querystring)

    return parse_search(response)


def extract_search(response):
    """extract json data from search page"""
    sel = Selector(response.text)
    # find script with page data in it
    script_with_data = sel.xpath('//script[contains(text(),"window.runParams")]')
    # select page data from javascript variable in script tag using regex
    return json.loads(script_with_data.re(r"window.runParams\s*=\s*({.+?});")[0])
def parse_search(response):
    """Parse search page response for product preview results"""
    data = extract_search(response)
    parsed = []
    for result in data["mods"]["itemList"]["content"]:
        parsed.append(
            {
                "id": result["productId"],
                "url": f"FILL",
                "type": result["productType"],  # can be either natural or ad
                "title": result["title"]["displayTitle"],
                "price": result["prices"]["salePrice"]["minPrice"],
                "currency": result["prices"]["salePrice"]["currencyCode"],
                "trade": result.get("trade", {}).get("tradeDesc"),  # trade line is not always present
                "thumbnail": result["image"]["imgUrl"].lstrip("/"),
                "store": {
                    "url": result["store"]["storeUrl"],
                    "name": result["store"]["storeName"],
                    "id": result["store"]["storeId"],
                    "ali_id": result["store"]["FILL"],
                },
            }
        )
    return parsed

def getProduct(prodId):
    url = "FILL"+str(prodId)+".html"

    payload = ""
    headers = {
        "cookie": "ali_apache_id=33.3.24.190.164855992375.189351.0; XSRF-TOKEN=cebdb283-e098-462b-b5ca-8e7d56368f8a; intl_locale=en_US; xman_f=T00k0g+45+kVyCBXFo+ZGxD4zF9tzVgSvWgXd6Xs6lO7PToQcZ9cOUzrlujs980cc0aNf8b7EtN8yHSnFx4i+JKTnNxoZtVNEoKtF7UwYYJim3Dknb8ZAw==; aep_usuc_f=site=glo&c_tp=DKK&ups_d=0|0|0|0&ups_u_t=1664731016912&region=DK&b_locale=en_US&ae_u_p_s=1; acs_usuc_t=x_csrf=1b0xyd1bhfel9&acs_rt=c8e9ec21ee7443f5a0b657a018169e3f; xman_t=g/CiU0iYUAq/arD8sEi676y5pnqn5FXpwnODC+lXbwNOKECkJCe1oxojkz6oDABz; ali_apache_track=; ali_apache_tracktmp=; e_id=pt90; _m_h5_tk=cbdd747895668e3e0f3b4ff94aa2acb8_1663589691466; _m_h5_tk_enc=b52d006eda32cbd4eefda76070103880; AKA_A2=A; xman_us_f=x_locale=en_US&x_l=0&x_c_chg=0&x_as_i=%7B%22cookieCacheEffectTime%22%3A1663588459153%2C%22isCookieCache%22%3A%22Y%22%2C%22ms%22%3A%220%22%7D&acs_rt=c8e9ec21ee7443f5a0b657a018169e3f; aep_history=keywords%5E%0Akeywords%09%0A%0Aproduct_selloffer%5E%0Aproduct_selloffer%091005004727409574%091005003490461568%094000094534783%094001082200193%0932946165416%091005004689057643%091005004023414962%091005002227573759; tfstk=cQENBbZQRUKQdZZXCDo4YFLzrgoOZ3lm6wliS_hFe5tvFvuGiNRxKd1gTJv4hVf..; l=eBxlqkMILY9NzvM9BOfwourza77OSIRAguPzaNbMiOCPONfp5tBAB6oTkq89C3MNh6bHR3PgPMOyBeYBY3tSnxv9gJLRMsHmn; isg=BO_vs6TH0E6Vp9UZ9pp8Pombfgr5lEO269bNHwF8i95lUA9SCWTTBu0A0ljuKBsu; JSESSIONID=8AD018BB354E2640785C6ABB3C61560F; intl_common_forever=Olgl8BsJYFf/7O5KYJ0oCb86XijI/KbH84eWvOEK6QEEqUFy8AhMlg==",
        "authority": "FILL",
        "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8",
        "accept-language": "en-US,en;q=0.5",
        "cache-control": "max-age=0",
        "sec-fetch-dest": "document",
        "sec-fetch-mode": "navigate",
        "sec-fetch-site": "same-origin",
        "sec-fetch-user": "?1",
        "sec-gpc": "1",
        "upgrade-insecure-requests": "1",
        "user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36"
    }

    response = requests.request("GET", url, data=payload, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    links = soup.find("div", {"class" : "breadcrumb"}).find_all('a')
    return [link.text for link in links]

#cat = getCate()
#cat = cat[:5]

### Set this value to the file you are scraping
scrapefile = "FILL"

datafile = open(scrapefile,'r')
res = json.load(datafile)
datafile.close()

for link in res["todo"]:
    print(link)
    for page in range(1, 10):
        try:
            products = getCatPage(link, page)
        except:
            print("Problem with loading catagory!!!!")
            break
        if len(products) < 60: break

        for prod in products:
            if prod['url'] in res['data'].keys(): continue
            try: labels = getProduct(prod['id'])
            except:
                print("Problem with loading product!!!!")
                continue
            if len(labels) != 1: res["data"][prod['url']] = labels
            delay = randint(1,5)
            print("Product Sleep " + str(delay) + "s")
            sleep(delay)
        try:
            with open(scrapefile, 'w') as f:
                json.dump(res, f, indent = 6)
        except:
            with open(scrapefile, 'w') as f:
                json.dump(res, f, indent = 6)
            quit()

    res["todo"].remove(link)
    try:
        with open(scrapefile, 'w') as f:
            json.dump(res, f, indent = 6)
    except:
        with open(scrapefile, 'w') as f:
            json.dump(res, f, indent = 6)
        quit()
    delay = randint(1,5)
    print("Page Sleep " + str(delay) + "s")
    sleep(delay)

print("Scrape done move to next file")

Product Sleep 3s
Product Sleep 2s
Product Sleep 5s
Product Sleep 5s
Product Sleep 4s
Product Sleep 3s
Product Sleep 2s


KeyboardInterrupt: ignored

# Merger

In [ ]:
import json

#do this once you are ready to do the next scrape file
scrapefile1 = "FILL" #The scrape with empty todo
scrapefile2 = "FILL" #the scrape with filled todo

datafile = open(scrapefile1,'r')
scrape1 = json.load(datafile)
datafile.close()

datafile = open(scrapefile2,'r')
scrape2 = json.load(datafile)
datafile.close()

scrape2["data"] = {**scrape1["data"],  **scrape2["data"]}


with open(scrapefile2, 'w') as f:
            json.dump(scrape2, f, indent = 6)